In [1]:
import numpy as np
import time

## ベクトルの規格化（結果のみ）
$n \in \{ 0, 1, \dots , N-1\}$に対し，$\boldsymbol{a}_n \in \mathbb{R}^K$とする．

このとき，$n \in \{ 0, 1, \dots , N-1\}$に対し，
\begin{align}
\boldsymbol{a}_n \leftarrow \frac{\boldsymbol{a}_n}{\sum_{k=0}^{K-1}a_{n,k}}
\end{align}
を計算したい．

In [2]:
# データ生成
N = 10000
K = 10

A = np.random.rand(N,K)
A_copy = np.copy(A)

In [3]:
# for文による規格化
start = time.time()

tmp = 0.0
for n in range(N):
    tmp = 0.0
    for k in range(K):
        tmp += A[n,k]
    for k in range(K):
        A[n,k] /= tmp

end = time.time()

print(end - start)

0.18643975257873535


In [4]:
# for文を使わない規格化
start = time.time()

A_copy = A_copy / A_copy.sum(axis=1)[:,np.newaxis]
# A_copy = A_copy / A_copy.sum(axis=1,keepdims=True)

end = time.time()

print(end - start)

0.005480527877807617


In [5]:
# 同じ計算になっていることを確認
print(np.sum(np.abs(A - A_copy)))

5.165647742183792e-13


## shape
行列のサイズに相当する概念だが，スカラーと1次元ベクトルと1×1行列は区別する．

In [6]:
A = np.array([[1,2,3],
              [4,5,6]]) # ちなみにこの宣言方法だと自動的に整数型になってしまうので注意
print(A.shape)

(2, 3)


In [7]:
B = np.array([[[1,2,3],
               [4,5,6]],

              [[7,8,9],
               [10,11,12]]])
print(B.shape)

(2, 2, 3)


In [8]:
a_1 = np.ones(1) # こちらは実数型で宣言される
a_2 = np.ones((1,1))
a_3 = np.ones((1,1,1))

print(a_1)
print(a_1.shape)
print(a_2)
print(a_2.shape)
print(a_3)
print(a_3.shape)

[1.]
(1,)
[[1.]]
(1, 1)
[[[1.]]]
(1, 1, 1)


## Broadcast
→スライドで説明

## ベクトルの規格化（仕組み）
$n \in \{ 0, 1, \dots , N-1\}$に対し，$\boldsymbol{a}_n \in \mathbb{R}^K$とする．

このとき，$n \in \{ 0, 1, \dots , N-1\}$に対し，
\begin{align}
\boldsymbol{a}_n \leftarrow \frac{\boldsymbol{a}_n}{\sum_{k=0}^{K-1}a_{n,k}}
\end{align}
を計算したい．

In [9]:
# データ生成
N = 10000
K = 10

A_for = np.random.rand(N,K)
A_1 = np.copy(A_for)
A_2 = np.copy(A_for)

In [10]:
# for文のみ
start = time.time()

tmp = 0.0
for n in range(N):
    tmp = 0.0
    for k in range(K):
        tmp += A_for[n,k]
    for k in range(K):
        A_for[n,k] /= tmp

end = time.time()

print(end - start)

0.1540236473083496


In [11]:
# ブロードキャスト利用レベル1
start = time.time()

for n in range(N):
    A_1[n] /= A_1[n].sum()

end = time.time()

print(end - start)

print(A_1[0].shape)
print(A_1[0].sum().shape)

0.07667708396911621
(10,)
()


In [44]:
# # 失敗例
A_2 /= A_2.sum(axis=1)

ValueError: operands could not be broadcast together with shapes (10000,10) (10000,) (10000,10) 

In [13]:
# ブロードキャスト利用レベル2
start = time.time()

A_2 /= A_2.sum(axis=1)[:,np.newaxis]

end = time.time()

print(end - start)

0.0010004043579101562


In [14]:
# なぜうまくいくのか
print(A_2.sum(axis=1)[:,np.newaxis].shape)

(10000, 1)


In [15]:
# （スライス）＋（np.newaxisまたはNone）=（追加軸のサイズを1にしたreshape）
A = np.random.rand(3,4)
print(A.sum(axis=1).reshape(3,1))
print(A.sum(axis=1)[:,np.newaxis])
print(A.sum(axis=1)[:,None])

[[1.92467546]
 [1.9832042 ]
 [1.38556945]]
[[1.92467546]
 [1.9832042 ]
 [1.38556945]]
[[1.92467546]
 [1.9832042 ]
 [1.38556945]]


In [16]:
# np.sumやnp.maxのkeepdimsオプションをTrueにすると，和でつぶれる軸をサイズ1で残す
A = np.random.rand(2,3,4,5,6)
print(A.sum(axis=(1,4)).shape)
print(A.sum(axis=(1,4),keepdims=True).shape)

(2, 4, 5)
(2, 1, 4, 5, 1)


**より実践的な例**

* 任意の$n \in \{ 0, 1, \dots , N-1\}$に対して$\boldsymbol{a} \in \mathbb{R}^K$
* 任意の$n \in \{ 0, 1, \dots , N-1\}$に対して$\boldsymbol{b} \in [0, 1]^K$

このとき，任意の$n \in \{ 0, 1, \dots , N-1\}$に対して
\begin{align}
b_i = \frac{\exp (a_i)}{\sum_i \exp (a_i)}
\end{align}
を計算したい．しかし，このまま計算すると$\exp (a_i)$の計算でオーバーフローする可能性がある．（$b_i$全体としては$[0, 1]$に収まるのでオーバーフローすることはない）

そこで，$a_\mathrm{max} := \max_i a_i$とおいて，以下のようにするとオーバーフローを防げる．
\begin{align}
b_i = \frac{\exp (a_i - a_\mathrm{max})}{\sum_i \exp (a_i - a_\mathrm{max})}
\end{align}

In [17]:
# データ生成
N = 10000
K = 10

A_for = np.random.rand(N,K)
B_for = np.empty((N,K))

A_1 = np.copy(A_for)
B_1 = np.empty((N,K))

A_2 = np.copy(A_for)
B_2 = np.empty((N,K))

In [18]:
# for文のみ
start = time.time()

tmp_max = 1.0e+10
tmp_sum = 0.0
for n in range(N):
    tmp_max = 1.0e+10
    for k in range(K):
        if A_for[n,k] > tmp_max:
            tmp_max = A_for[n,k]
    
    for k in range(K):
        B_for[n,k] = np.exp(A_for[n,k] - tmp)
    
    tmp_sum = 0.0
    for k in range(K):
        tmp_sum += B_for[n,k]

    for k in range(K):
        B_for[n,k] /= tmp_sum
        
end = time.time()

print(end - start)

0.495164155960083


In [19]:
# ブロードキャスト利用レベル1
N = 10000
K = 10

start = time.time()

for n in range(N):
    B_1[n] = np.exp(A_1[n] - A_1[n].max())
    B_1[n] /= B_1[n].sum()

end = time.time()

print(end - start)

0.2061443328857422


In [20]:
# ブロードキャスト利用レベル2
N = 10000
K = 10

start = time.time()

B_2 = np.exp(A_2 - A_2.max(axis=1, keepdims=True))
B_2 /= B_2.sum(axis=1, keepdims=True)

end = time.time()

print(end - start)

0.009595870971679688


In [21]:
# 結果の確認
print(np.sum(np.abs(B_for - B_1)))
print(np.sum(np.abs(B_for - B_2)))

1.9971316267408667e-12
1.9965487596529385e-12


## 行列の和や積
**例1**
* $\boldsymbol{w} \in \mathbb{R}^N$
* 任意の$n \in \{ 0, 1, \dots , N-1\}$に対し，$A_n \in \mathbb{R}^{K \times M}$
* $\bar{A} \in \mathbb{R}^{K \times M}$

このとき，
\begin{align}
\bar{A} \leftarrow \sum_{n=0}^{N-1} w_n A_n
\end{align}
を計算したい．

In [22]:
# データ生成
N = 1000
K = 10
M = 20

w_0 = np.random.rand(N)
A_0 = np.random.rand(N,K,M)
A_bar_0 = np.zeros((K,M))

w_1 = np.copy(w_0)
A_1 = np.copy(A_0)
A_bar_1 = np.empty((K,M))

In [23]:
# ブロードキャスト利用レベル0
start = time.time()

for n in range(N):
    A_bar_0 += w_0[n] * A_0[n]

end = time.time()

print(end - start)

0.009620428085327148


In [24]:
# ブロードキャスト利用レベル1
start = time.time()

A_bar_1 = np.sum(w_1[:,np.newaxis,np.newaxis] * A_1, axis=0)

end = time.time()

print(end - start)

0.002535104751586914


In [25]:
# 結果の確認
print(np.sum(np.abs(A_bar_0 - A_bar_1)))

0.0


**行列積のブロードキャスト**<br>
→スライドで説明

**例2**
* 任意の$n \in \{ 0, 1, \dots , N-1\}$に対し，$A_n \in \mathbb{R}^{K \times M}$
* $B \in \mathbb{R}^{M \times K}$
* $\boldsymbol{t} \in \mathbb{R}^N$

このとき，任意の$n \in \{ 0, 1, \dots , N-1\}$に対し，
\begin{align}
t_n \leftarrow \mathrm{Tr} \{ A_n B \}
\end{align}
を計算したい．

In [78]:
# データの生成
N = 10000
K = 10
M = 20

A_for = np.random.rand(N,K,M)
B_for = np.random.rand(M,K)
t_for = np.empty(N)

A_1 = np.copy(A_for)
B_1 = np.copy(B_for)
t_1 = np.empty(N)

A_2 = np.copy(A_for)
B_2 = np.copy(B_for)
t_2 = np.empty(N)

In [66]:
# for文のみ
start = time.time()

for n in range(N):
    t_for[n] = np.trace(A_for[n] @ B_for)
    
end = time.time()

print(end - start)

0.14568448066711426


In [70]:
# ブロードキャスト利用レベル2
start = time.time()

t_1 = np.trace(A_1 @ B_1, axis1 = 1, axis2 = 2)

end = time.time()

print(end - start)

0.012549161911010742


In [60]:
# 結果の確認
print(np.sum(np.abs(t_for - t_1)))

0.0


In [79]:
# より高速なトレースの計算?
start = time.time()

t_2 = (A_2 * B_2.T).sum()
    
end = time.time()

print(end - start)

0.01363992691040039


In [83]:
np.einsum('ijk,ijk->', A_2, B_2[np.newaxis])

ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (10000,10,20)->(10000,10,20) (1,20,10)->(1,20,10) 

## 二次形式

* 任意の$k \in \{ 0, 1, \dots , K-1\}$と任意の$j \in \{ 0, 1, \dots , J-1\}$に対して，$\boldsymbol{x}_{k, j} \in \mathbb{R}^d$
* 任意の$k \in \{ 0, 1, \dots , K-1\}$に対して$\boldsymbol{\mu}_k \in \mathbb{R}^d$
* 任意の$l \in \{ 0, 1, \dots , L-1\}$に対して，$\boldsymbol{\Lambda}_l \in \mathbb{R}^{d \times d}$
* $\boldsymbol{c} \in \mathbb{R}^K$

このとき，任意の$k \in \{ 0, 1, \dots , K-1\}$に対して，
\begin{align}
c_k \leftarrow \sum_{j, l} (\boldsymbol{x}_{k, j} - \boldsymbol{\mu}_k)^\top \boldsymbol{\Lambda}_l (\boldsymbol{x}_{k, j} - \boldsymbol{\mu}_k)
\end{align}
を計算したい．

In [30]:
# データの生成
K = 100
J = 80
d = 10
L = 20

X_for = np.random.rand(K,J,d,1)
mu_for = np.random.rand(K,d,1)
Lambda_for = np.random.rand(L,d,d)
Lambda_for = (Lambda_for.transpose((0,2,1)) + Lambda_for) / 2.0
c_for = np.zeros((K,1))

X_1 = np.copy(X_for)
mu_1 = np.copy(mu_for)
Lambda_1 = np.copy(Lambda_for)
c_1 = np.empty((K,1))

X_2 = np.copy(X_for)
mu_2 = np.copy(mu_for)
Lambda_2 = np.copy(Lambda_for)
c_2 = np.empty((K,1))

X_3 = np.copy(X_for)
mu_3 = np.copy(mu_for)
Lambda_3 = np.copy(Lambda_for)
c_3 = np.empty((K,1))

In [31]:
# for文のみ
start = time.time()

for k in range(K):
    for j in range(J):
        for l in range(L):
            c_for[k,0] += ((X_for[k,j] - mu_for[k]).T
                           @ Lambda_for[l]
                           @ (X_for[k,j] - mu_for[k]))

end = time.time()

print(end - start)

2.312133550643921


In [32]:
# ブロードキャスト利用レベル3(?)
start = time.time()

c_1[:,0] = np.sum((X_1[:,:,np.newaxis,:,:] - mu_1[:,np.newaxis,np.newaxis,:,:]).transpose(0,1,2,4,3)
                   @ Lambda_1[np.newaxis,np.newaxis,:,:,:]
                   @ (X_1[:,:,np.newaxis,:,:] - mu_1[:,np.newaxis,np.newaxis,:,:]),
           axis=(1,2))[:,0,0]

end = time.time()

print(end - start)

0.07006120681762695


In [33]:
# ちょっとわかりやすくなるかもしれない考え方
start = time.time()

c_2[:,0] = np.sum(axis = 1,
                  a = np.sum(axis = 2,
                             a = (X_2[:,:,np.newaxis,:,:] - mu_2[:,np.newaxis,np.newaxis,:,:]).transpose(0,1,2,4,3)
                                  @ Lambda_2[np.newaxis,np.newaxis,:,:,:]
                                  @ (X_2[:,:,np.newaxis,:,:] - mu_2[:,np.newaxis,np.newaxis,:,:])
                  )
           )[:,0,0]

end = time.time()

print(end - start)

0.05057787895202637


In [34]:
# 結果の確認
print(c_for.shape)
print(c_1.shape)
print(np.sum(np.abs(c_for - c_1)))

(100, 1)
(100, 1)
1.1431211532908492e-10


In [35]:
# 結果の確認
print(c_2.shape)
print(np.sum(np.abs(c_for - c_2)))

(100, 1)
1.1448264558566734e-10


もっとうまい方法は
\begin{align}
c_k \leftarrow \sum_l \mathrm{Tr} \left\{ \boldsymbol{\Lambda}_l \sum_j(\boldsymbol{x}_{k, j} - \boldsymbol{\mu}_k) (\boldsymbol{x}_{k, j} - \boldsymbol{\mu}_k)^\top\right\}
\end{align}

In [36]:
# 数式の同値変形まで使う
start = time.time()

c_3[:,0] = np.sum(
                np.trace(
                    Lambda_3[np.newaxis,:,:,:]
                    @ np.sum((X_3[:,:,:,:] - mu_3[:,np.newaxis,:,:])
                          @ (X_3[:,:,:,:] - mu_3[:,np.newaxis,:,:]).transpose(0,1,3,2),
                      axis=1)[:,np.newaxis,:,:],
                axis1=2, axis2=3),
           axis=1)

end = time.time()

print(end - start)

0.018059730529785156


In [37]:
# 結果の確認
print(c_3.shape)
print(np.sum(np.abs(c_for - c_3)))

(100, 1)
1.1937117960769683e-10


## 条件を満たす要素の指定
ゼロ除算を防ぐため，以下のような処理を行いたい場合がある．

In [38]:
# データの生成
EPSILON = 1.0e-10

N = 10000
K = 100
A_for = np.random.rand(N, K)
A_1 = np.copy(A_for)

# for文のみ
start = time.time()

for i in range(N):
    for j in range(K):
        if A_for[i,j] < EPSILON:    # ゼロがあったら十分小さな数で置き換える
            A_for[i,j] = EPSILON

end = time.time()

print(end - start)

0.8077993392944336


これは，for文を使わずに以下のように書ける

In [39]:
# ブロードキャスト利用
start = time.time()

A_1[A_1 < EPSILON] = EPSILON

end = time.time()

print(end-start)

0.005132436752319336


詳しい挙動は以下

In [40]:
A = np.array([[1,2,3],
              [4,5,6]])
print(A > 2)

[[False False  True]
 [ True  True  True]]


In [41]:
B = np.array([[False,False,True],
              [True,True,True]])
print(A[B])

[3 4 5 6]


## おまけ（研究での利用例）

各$k \in \{1, 2, \dots , K \}$に対し
\begin{align}
\boldsymbol{W}^{-1}_k &= \boldsymbol{W}^{-1} + \sum_{j, m, n} \hat{\pi}_{mn, k}^{(j)} \left( \boldsymbol{\Lambda}_{mn}^{(j)} \right)^{-1} + \sum_{j, m, n} \tilde{\pi}_{mn, k}^{(j)} \left( \hat{\boldsymbol{\mu}}_{mn}^{(j)} - \tilde{\boldsymbol{\mu}}_k \right) \left( \hat{\boldsymbol{\mu}}_{mn}^{(j)} - \tilde{\boldsymbol{\mu}}_k \right)^\top + \frac{\beta \tilde{N}_k}{\beta + \tilde{N}_k} \left( \tilde{\boldsymbol{\mu}}_k - \eta \right) \left( \tilde{\boldsymbol{\mu}}_k - \eta \right)^\top
\end{align}
（$j, m, n$はプログラム上では区別してません）

In [42]:
# def update_W_hat(self):
#     self.W_hat_inv = (W_INV
#                       + np.sum(self.pi_hat[:,:,np.newaxis,np.newaxis]*self.Lambda_hat_inv[:,np.newaxis,:,:],axis=0)
#                       + np.sum(self.pi_tilde[:,:,np.newaxis,np.newaxis]
#                                 * (self.mu_hat[:,np.newaxis,:,:]-self.mu_tilde[np.newaxis,:,:,:])
#                                 @ (self.mu_hat[:,np.newaxis,:,:]-self.mu_tilde[np.newaxis,:,:,:]).transpose(0,1,3,2)
#                                 ,axis=0)
#                       + (BETA*self.N_tilde/self.beta_hat)[:,np.newaxis,np.newaxis]
#                           * (self.mu_tilde-ETA)
#                           @ (self.mu_tilde-ETA).transpose(0,2,1)
#                       )
#     self.W_hat = np.linalg.inv(self.W_hat_inv)